In [1]:
import pandas as pd
import json
import re
import os
import glob
from unidecode import unidecode
import requests
from pandasql import sqldf

----

VRA data

In [2]:
#Load VRA data

df_vra = pd.DataFrame()

for j in glob.glob('VRA/*.json'):
    #Get json file
    with open(j, encoding='utf-8-sig') as file:
        json_vra = json.load(file)
        
    #Fetch it to pandas DataFrame
    df_loop_vra = pd.DataFrame(json_vra)
    print('Number of rows in round {} is {}'.format(j, df_loop_vra.shape[0]))
    
    #Gather all json files into one df
    df_vra = pd.concat([df_vra,df_loop_vra])

print('----\nTotal #rows: ', df_vra.shape[0])

#Could be a function as well


Number of rows in round VRA/VRA_20211.json is 58550
Number of rows in round VRA/VRA_202110.json is 59430
Number of rows in round VRA/VRA_202111.json is 64004
Number of rows in round VRA/VRA_20212.json is 43692
Number of rows in round VRA/VRA_20213.json is 39514
Number of rows in round VRA/VRA_20214.json is 29122
Number of rows in round VRA/VRA_20215.json is 35750
Number of rows in round VRA/VRA_20216.json is 40955
Number of rows in round VRA/VRA_20217.json is 53313
Number of rows in round VRA/VRA_20218.json is 54928
Number of rows in round VRA/VRA_20219.json is 56545
----
Total #rows:  535803


In [3]:
df_vra.head()

,ICAOEmpresaAérea,NúmeroVoo,CódigoAutorização,CódigoTipoLinha,ICAOAeródromoOrigem,ICAOAeródromoDestino,PartidaPrevista,PartidaReal,ChegadaPrevista,ChegadaReal,SituaçãoVoo,CódigoJustificativa
0,MWM,9702,7,C,SBBE,SBEG,2021-01-19 12:35:00,2021-01-19 17:17:00,2021-01-19 14:45:00,2021-01-19 19:22:00,REALIZADO,N/A
1,MWM,9701,7,C,SBGL,SBPA,2021-01-26 10:00:00,2021-01-26 09:55:00,2021-01-26 12:05:00,2021-01-26 12:04:00,REALIZADO,N/A
2,MWM,9701,7,C,SBEG,SBBE,2021-01-18 21:15:00,2021-01-18 22:35:00,2021-01-18 23:20:00,2021-01-19 00:50:00,REALIZADO,N/A
3,MWM,9700,0,C,SBGL,SBEG,2021-01-14 20:30:00,2021-01-14 20:27:00,2021-01-15 00:45:00,2021-01-15 00:39:00,REALIZADO,N/A
4,MWM,9700,7,C,SBKP,SBEG,2021-01-24 11:30:00,2021-01-24 19:35:00,2021-01-24 15:10:00,2021-01-24 23:21:00,REALIZADO,N/A


In [4]:
#A new function is needed to ajust headers

def to_snake_case(header_list):
    '''Convert camel case notation to snake one'''
    
    snake_case_list = []
    for col in header_list:
        #Check if there is space
        if ' ' in col:
            col = col.replace(' ','')

        new_col = re.sub('(.)([A-Z][a-z]+)', r'\1_\2', col)
        new_col = re.sub('([a-z0-9])([A-Z])', r'\1_\2', new_col).lower()
        
        #Take off accent mark from new_col
        snake_case_list.append(unidecode(new_col))
    
    return snake_case_list


In [5]:
#Header ajust

df_vra.columns = to_snake_case(df_vra.columns.to_list())


In [6]:
df_vra.head()

,icao_empresa_aerea,numero_voo,codigo_autorizacao,codigo_tipo_linha,icao_aerodromo_origem,icao_aerodromo_destino,partida_prevista,partida_real,chegada_prevista,chegada_real,situacao_voo,codigo_justificativa
0,MWM,9702,7,C,SBBE,SBEG,2021-01-19 12:35:00,2021-01-19 17:17:00,2021-01-19 14:45:00,2021-01-19 19:22:00,REALIZADO,N/A
1,MWM,9701,7,C,SBGL,SBPA,2021-01-26 10:00:00,2021-01-26 09:55:00,2021-01-26 12:05:00,2021-01-26 12:04:00,REALIZADO,N/A
2,MWM,9701,7,C,SBEG,SBBE,2021-01-18 21:15:00,2021-01-18 22:35:00,2021-01-18 23:20:00,2021-01-19 00:50:00,REALIZADO,N/A
3,MWM,9700,0,C,SBGL,SBEG,2021-01-14 20:30:00,2021-01-14 20:27:00,2021-01-15 00:45:00,2021-01-15 00:39:00,REALIZADO,N/A
4,MWM,9700,7,C,SBKP,SBEG,2021-01-24 11:30:00,2021-01-24 19:35:00,2021-01-24 15:10:00,2021-01-24 23:21:00,REALIZADO,N/A


In [7]:
#Save data

df_vra.to_csv('RESPOSTAS/VRA_final.csv')


----

AIR_CIA data

In [8]:
#Load AIR_CIA data

df_air_cia = pd.DataFrame()

for csv in glob.glob('AIR_CIA/*.csv'):    
    #Fetch it to pandas DataFrame
    df_loop_air_cia = pd.read_csv(csv, sep=';')
    print('Number of rows in round {} is {}'.format(csv, df_loop_air_cia.shape[0]))
    
    #Gather all csv files into one df
    df_air_cia = pd.concat([df_air_cia,df_loop_air_cia])

print('----\nTotal #rows: ', df_air_cia.shape[0])

#Could be a function as well


Number of rows in round AIR_CIA/ANAC_20211220_203627.csv is 13
Number of rows in round AIR_CIA/ANAC_20211220_203643.csv is 5
Number of rows in round AIR_CIA/ANAC_20211220_203733.csv is 2
----
Total #rows:  20


In [9]:
df_air_cia.head()

,Razão Social,ICAO IATA,CNPJ,Atividades Aéreas,Endereço Sede,Telefone,E-Mail,Decisão Operacional,Data Decisão Operacional,Validade Operacional
0,ABSA - AEROLINHAS BRASILEIRAS S.A.,LTG M3,00.074.635/0001-33,TRANSPORTE AÉREO REGULAR,"AEROPORTO INTERNACIONAL DE VIRACOPOS, RODOVIA ...",(11) 5582-8055,gar@tam.com.br,DECISÃO Nº 41,22/04/2015,23/04/2025
1,AEROSUL TÁXI AÉREO LTDA (EX.: AUSTEN TÁXI AÉRE...,ASO 2S,27.315.694/0001-02,"SERVIÇOS AÉREOS PÚBLICOS, TÁXI-AÉREO, TRANSPOR...","RODOVIA PR 218, KM 7, JARDIM UNIVERSITÁRIO, AE...",(43) 3176-4030,operacoes@aerosul.com.br,DECISÃO Nº 282,10/02/2021,NaN
2,ASTA LINHAS AÉREAS LTDA ( EX - AMÉRICA DO SUL ...,SUL 0A,12.703.737/0001-50,"SERVIÇOS AÉREOS PÚBLICOS, TRANSPORTE AÉREO REG...","AV GOVERNADOR JOÃO PONCE DE ARRUDA, S/N, HANGA...",(65) 3614-2684 / (18) 2104-1000,NaN,DECISÃO Nº 115,31/07/2019,02/08/2024
3,ATA - AEROTÁXI ABAETÉ LTDA.,ABJ,14.674.451/0001-19,"TÁXI-AÉREO, TRANSPORTE AÉREO NÃO REGULAR, TRAN...","PÇA. GAGO COUTINHO, S/N, AEROPORTO INTERNACIO...",(71) 34629600 / (71) 996643563,guilherme@abaete.com.br,DECISÃO Nº 36,05/03/2020,NaN
4,AZUL CONECTA LTDA. (EX TWO TAXI AEREO LTDA),ACN 2F,04.263.318/0001-16,TRANSPORTE AÉREO REGULAR,"AVENIDA EMILIO ANTONON, 901, BAIRRO CHÁCARA AE...",(11) 45822355/ (11) 987433988,NaN,"Decisão nº 14, de 25/01/2018",25/01/2018,29/01/2028


In [10]:
#Split column ICAO IATA

df_air_cia['ICAO'], df_air_cia['IATA'] = df_air_cia['ICAO IATA'].str.split(' ', 1).str


/tmp/ipykernel_222/3692144492.py:3: FutureWarning: Columnar iteration over characters will be deprecated in future releases.
  df_air_cia['ICAO'], df_air_cia['IATA'] = df_air_cia['ICAO IATA'].str.split(' ', 1).str


In [11]:
#Order AIR_CIA columns

cols = df_air_cia.columns.to_list()
cols = [cols[0], cols[-2], cols[-1], *cols[2:-2]]

df_air_cia = df_air_cia[cols]

#Some IATA datapoints are NaN, pandas understands it as null value then I chose to keep it that way


In [12]:
#Header ajust

df_air_cia.columns = to_snake_case(df_air_cia.columns.to_list())


In [13]:
df_air_cia.head()

,razao_social,icao,iata,cnpj,atividades_aereas,endereco_sede,telefone,e-_mail,decisao_operacional,data_decisao_operacional,validade_operacional
0,ABSA - AEROLINHAS BRASILEIRAS S.A.,LTG,M3,00.074.635/0001-33,TRANSPORTE AÉREO REGULAR,"AEROPORTO INTERNACIONAL DE VIRACOPOS, RODOVIA ...",(11) 5582-8055,gar@tam.com.br,DECISÃO Nº 41,22/04/2015,23/04/2025
1,AEROSUL TÁXI AÉREO LTDA (EX.: AUSTEN TÁXI AÉRE...,ASO,2S,27.315.694/0001-02,"SERVIÇOS AÉREOS PÚBLICOS, TÁXI-AÉREO, TRANSPOR...","RODOVIA PR 218, KM 7, JARDIM UNIVERSITÁRIO, AE...",(43) 3176-4030,operacoes@aerosul.com.br,DECISÃO Nº 282,10/02/2021,NaN
2,ASTA LINHAS AÉREAS LTDA ( EX - AMÉRICA DO SUL ...,SUL,0A,12.703.737/0001-50,"SERVIÇOS AÉREOS PÚBLICOS, TRANSPORTE AÉREO REG...","AV GOVERNADOR JOÃO PONCE DE ARRUDA, S/N, HANGA...",(65) 3614-2684 / (18) 2104-1000,NaN,DECISÃO Nº 115,31/07/2019,02/08/2024
3,ATA - AEROTÁXI ABAETÉ LTDA.,ABJ,NaN,14.674.451/0001-19,"TÁXI-AÉREO, TRANSPORTE AÉREO NÃO REGULAR, TRAN...","PÇA. GAGO COUTINHO, S/N, AEROPORTO INTERNACIO...",(71) 34629600 / (71) 996643563,guilherme@abaete.com.br,DECISÃO Nº 36,05/03/2020,NaN
4,AZUL CONECTA LTDA. (EX TWO TAXI AEREO LTDA),ACN,2F,04.263.318/0001-16,TRANSPORTE AÉREO REGULAR,"AVENIDA EMILIO ANTONON, 901, BAIRRO CHÁCARA AE...",(11) 45822355/ (11) 987433988,NaN,"Decisão nº 14, de 25/01/2018",25/01/2018,29/01/2028


In [14]:
#Check duplicates

df_air_cia.razao_social.value_counts()


SIDERAL LINHAS AÉREAS LTDA.                                           2
ATA - AEROTÁXI ABAETÉ LTDA.                                           2
AEROSUL TÁXI AÉREO LTDA (EX.: AUSTEN TÁXI AÉREO LTDA)                 2
ABSA - AEROLINHAS BRASILEIRAS S.A.                                    1
RIMA - RIO MADEIRA AEROTÁXI LTDA.                                     1
PRIME TÁXI AIR LTDA                                                   1
OMNI TÁXI AÉREO S.A.                                                  1
TOTAL LINHAS AÉREAS S.A.                                              1
TAM LINHAS AÉREAS S.A.                                                1
MODERN TRANSPORTE AEREO DE CARGA S.A                                  1
PASSAREDO TRANSPORTES AÉREOS S.A.                                     1
MAP TRANSPORTES AÉREOS LTDA.                                          1
GOL LINHAS AÉREAS S.A. (EX- VRG LINHAS AÉREAS S.A.)                   1
AZUL LINHAS AÉREAS BRASILEIRAS S/A                              

In [15]:
#Drop duplicates

df_air_cia.drop_duplicates(inplace=True)


In [16]:
#### Save data

df_air_cia.to_csv('RESPOSTAS/AIR_CIA_final.csv')


----

AERODROMOS data

In [17]:
#A new function is defined just to easy API fecth

def get_aerodromos_API(list_of_sort, x_key= '38848842f4msha445764916f5ea9p1cf839jsn748a22044f77', url="https://airport-info.p.rapidapi.com/airport"):
    '''Receive a list of airports sort name, login key and airport API url. Return a list of dictionaries full of info and a list of airports not found'''
    
    headers = {
    "X-RapidAPI-Key": x_key,
    "X-RapidAPI-Host": "airport-info.p.rapidapi.com"
        }
    
    final_list = []
    aero_not_found = 0
    l_aero_not_found = []
    for icao in list_of_sort:
        querystring = {"icao":icao}
        response = requests.request("GET", url, headers=headers, params=querystring)
        
        #Check if aero has info
        if response.text == '{"error":{"text":"No airport found"}}\n':
            aero_not_found +=1
            l_aero_not_found.append(icao)
            print(icao, ' not found')
        
        #Convert to json object
        json_object = json.loads(response.text)
        
        #Gather all aeros
        final_list.append(json_object)
    
    print("\n# {} airport(s) not found".format(aero_not_found))
    
    return final_list, l_aero_not_found
        

In [18]:
#Fetch from API infos to pandas

all_aeros = set(df_vra['icao_aerodromo_origem'])
json_aeros, aeros_not_found = get_aerodromos_API(all_aeros)
df_aerodromos = pd.DataFrame(json_aeros)


UCFM  not found
SSTE  not found
SNTS  not found
SSCN  not found
SDTK  not found
KAUH  not found
SDNM  not found
SBDO  not found
SNRJ  not found
SPJC  not found
SNEB  not found
SSSB  not found
SBJI  not found
SBSI  not found
SBPG  not found
SDAG  not found
SBSO  not found
SBAC  not found
SNHS  not found
SNYA  not found
SBUY  not found
SNEE  not found
SBVC  not found
SBCN  not found
SSSC  not found
SBJD  not found
SNTI  not found
SYEC  not found
SBPO  not found
SWPH  not found
SBRD  not found
KPVG  not found
SBJE  not found
SDIM  not found
SJHG  not found
SWHP  not found

# 36 airport(s) not found


In [19]:
#Keep not null data

df_aerodromos = df_aerodromos[~df_aerodromos.icao.isnull()]


In [20]:
df_aerodromos.head()

,error,id,iata,icao,name,location,street_number,street,city,county,state,country_iso,country,postal_code,phone,latitude,longitude,uct,website
1,NaN,4565.0,MDE,SKRG,José María Córdova International Airport,"Medellín, Colombia",,,,Rionegro,Antioquia,CO,Colombia,54047,+57 4 4025110,6.170763,-75.427620,-300.0,http://www.aeropuertorionegro.co/
2,NaN,1200.0,CCS,SVMI,Simón Bolívar International Airport,"Caracas, Venezuela",,,Maiquetía,Vargas,Vargas,VE,Venezuela,1162,+58 424-2067470,10.597259,-67.005340,-240.0,http://www.aeropuerto-maiquetia.com.ve/
3,NaN,7845.0,URG,SBUG,Rubem Berta International Airport,"Uruguaiana, Rio Grande do Sul, Brazil",s/n,Acesso Marechal Setembrino de Carvalho,,Uruguaiana,Rio Grande do Sul,BR,Brazil,97513-780,+55 55 3413-1314,-29.783995,-57.035700,-180.0,http://www4.infraero.gov.br/aeroportos/aeropor...
4,NaN,4435.0,LVR,SILC,Bom Futuro Municipal Airport[2],"Lucas do Rio Verde, Mato Grosso, Brazil",s/n,Rod MT-449 KM 4,,Lucas do Rio Verde,Mato Grosso,BR,Brazil,78455-000,,-13.034962,-55.942123,-240.0,
5,NaN,2695.0,GRU,SBGR,São Paulo–Guarulhos International Airport,"São Paulo, Brazil",s/nº,Rod. Hélio Smidt,,Guarulhos,São Paulo,BR,Brazil,07190-100,+55 11 2445-2945,-23.430573,-46.473040,-180.0,http://www.gru.com.br/


In [21]:
#Check duplicates

df_aerodromos.id.value_counts()

#Lenght is equal #rows then there are not duplicates

4565.0    1
1399.0    1
6679.0    1
6003.0    1
199.0     1
         ..
2150.0    1
8874.0    1
6992.0    1
3473.0    1
1481.0    1
Name: id, Length: 265, dtype: int64

In [22]:
#Save data

df_aerodromos.to_csv('RESPOSTAS/AERODROMOS_final.csv')


---

Some views

In [23]:
df_vra.columns

Index(['icao_empresa_aerea', 'numero_voo', 'codigo_autorizacao',
       'codigo_tipo_linha', 'icao_aerodromo_origem', 'icao_aerodromo_destino',
       'partida_prevista', 'partida_real', 'chegada_prevista', 'chegada_real',
       'situacao_voo', 'codigo_justificativa'],
      dtype='object')

In [24]:
df_air_cia.columns

Index(['razao_social', 'icao', 'iata', 'cnpj', 'atividades_aereas',
       'endereco_sede', 'telefone', 'e-_mail', 'decisao_operacional',
       'data_decisao_operacional', 'validade_operacional'],
      dtype='object')

In [25]:
df_aerodromos.columns

Index(['error', 'id', 'iata', 'icao', 'name', 'location', 'street_number',
       'street', 'city', 'county', 'state', 'country_iso', 'country',
       'postal_code', 'phone', 'latitude', 'longitude', 'uct', 'website'],
      dtype='object')

In [26]:
#Lambda function is created just to write queries easier

pysqldf = lambda q: sqldf(q, globals())


In [27]:
#Quantidade de voo por rota de razão social válida

q = """
    
            SELECT 
                ac.razao_social, v.icao_aerodromo_origem, v.icao_aerodromo_destino, count(*) AS quantidade_voo
            FROM df_vra AS v
            LEFT JOIN df_air_cia ac ON v.icao_empresa_aerea = ac.icao
            WHERE ac.razao_social IS NOT NULL AND v.icao_aerodromo_origem = "SBEG"
            GROUP BY ac.razao_social, v.icao_aerodromo_origem, v.icao_aerodromo_destino
        
        ;
    """

pysqldf(q)


,razao_social,icao_aerodromo_origem,icao_aerodromo_destino,quantidade_voo
0,ABSA - AEROLINHAS BRASILEIRAS S.A.,SBEG,SBBR,10
1,ABSA - AEROLINHAS BRASILEIRAS S.A.,SBEG,SBGR,243
2,ABSA - AEROLINHAS BRASILEIRAS S.A.,SBEG,SBKP,4
3,ABSA - AEROLINHAS BRASILEIRAS S.A.,SBEG,SEQM,136
4,ABSA - AEROLINHAS BRASILEIRAS S.A.,SBEG,SKBO,46
...,...,...,...,...
71,TAM LINHAS AÉREAS S.A.,SBEG,SBGR,1086
72,TAM LINHAS AÉREAS S.A.,SBEG,SBKP,1
73,TAM LINHAS AÉREAS S.A.,SBEG,SBPV,18
74,TAM LINHAS AÉREAS S.A.,SBEG,SPJC,1


In [28]:
#"Para cada companhia aérea trazer a rota mais utilizada"

q1 = """
        WITH rotas AS (
            SELECT 
                ac.razao_social,
                ac.icao, 
                v.icao_aerodromo_origem, 
                v.icao_aerodromo_destino, 
                count(*) AS quantidade_voo
                
            FROM df_vra AS v
            LEFT JOIN df_air_cia ac ON v.icao_empresa_aerea = ac.icao
            WHERE ac.razao_social IS NOT NULL
            GROUP BY ac.icao, ac.razao_social, v.icao_aerodromo_origem, v.icao_aerodromo_destino
        ),
        
            row_table AS(
            SELECT *, 
                ROW_NUMBER() OVER(PARTITION BY razao_social ORDER BY quantidade_voo DESC) AS row_id

            FROM rotas
        ),
        
            df_aerodromos2 AS(
            SELECT 
                icao as icao2,
                name as name2,
                state as state2
                
            FROM df_aerodromos  
        )
        
        SELECT 
            rw.row_id,
            rw.razao_social,
            
            ae_o.name AS nome_aero_origem,
            ae_o.icao AS icao_aero_origem,
            ae_o.state AS estado_aero_origem,
            
            ae_d.name2 AS nome_aero_destino,
            ae_d.icao2 AS icao_aero_destino,
            ae_d.state2 AS estado_aero_destino,
            rw.quantidade_voo
        
        FROM row_table AS rw
        LEFT JOIN df_aerodromos ae_o ON rw.icao_aerodromo_origem = ae_o.icao
        LEFT JOIN df_aerodromos2 ae_d ON rw.icao_aerodromo_destino = ae_d.icao2
        WHERE rw.row_id =1
        
        ;
    """

pysqldf(q1)

#We'he got 17 air companies, but only 13 appears. Because 5 do not have any fly registred


,row_id,razao_social,nome_aero_origem,icao_aero_origem,estado_aero_origem,nome_aero_destino,icao_aero_destino,estado_aero_destino,quantidade_voo
0,1,ABSA - AEROLINHAS BRASILEIRAS S.A.,Mariscal Sucre International Airport,SEQM,Pichincha,Miami International Airport,KMIA,Florida,377
1,1,AEROSUL TÁXI AÉREO LTDA (EX.: AUSTEN TÁXI AÉRE...,Hercílio Luz International Airport,SBFL,Santa Catarina,Caçador Airport,SBCD,Santa Catarina,26
2,1,ASTA LINHAS AÉREAS LTDA ( EX - AMÉRICA DO SUL ...,Marechal Rondon International Airport,SBCY,Mato Grosso,Juína Airport,SWJN,State of Mato Grosso,61
3,1,AZUL CONECTA LTDA. (EX TWO TAXI AEREO LTDA),Coari Airport,SWKO,State of Amazonas,Eduardo Gomes International Airport,SBEG,Amazonas,34
4,1,AZUL LINHAS AÉREAS BRASILEIRAS S/A,Viracopos/Campinas International Airport,SBKP,São Paulo,Santos Dumont Airport,SBRJ,Rio de Janeiro,2540
5,1,GOL LINHAS AÉREAS S.A. (EX- VRG LINHAS AÉREAS ...,Santos Dumont Airport,SBRJ,Rio de Janeiro,São Paulo–Congonhas Airport,SBSP,São Paulo,2514
6,1,MAP TRANSPORTES AÉREOS LTDA.,Eduardo Gomes International Airport,SBEG,Amazonas,None,None,None,480
7,1,MODERN TRANSPORTE AEREO DE CARGA S.A,Viracopos/Campinas International Airport,SBKP,São Paulo,Brasília International Airport (Presidente J. ...,SBBR,Distrito Federal,162
8,1,OMNI TÁXI AÉREO S.A.,Santa Maria Airport,SBAR,California,Rio de Janeiro–Galeão International Airport,SBGL,Rio de Janeiro,15
9,1,PASSAREDO TRANSPORTES AÉREOS S.A.,Porto Seguro Airport,SBPS,Bahia,Deputado Luís Eduardo Magalhães International ...,SBSV,Bahia,724


In [29]:
#None flies resgistred to those companies:

none_reg_co = ['RIMA - RIO MADEIRA AEROTÁXI LTDA.', 'PRIME TÁXI AIR LTDA', 
                'ATA - AEROTÁXI ABAETÉ LTDA.','RTS - ROTA DO SOL TÁXI AÉREO LTDA.']

df_air_cia[df_air_cia.razao_social.isin(none_reg_co)]

#*Continue at next cell


,razao_social,icao,iata,cnpj,atividades_aereas,endereco_sede,telefone,e-_mail,decisao_operacional,data_decisao_operacional,validade_operacional
3,ATA - AEROTÁXI ABAETÉ LTDA.,ABJ,NaN,14.674.451/0001-19,"TÁXI-AÉREO, TRANSPORTE AÉREO NÃO REGULAR, TRAN...","PÇA. GAGO COUTINHO, S/N, AEROPORTO INTERNACIO...",(71) 34629600 / (71) 996643563,guilherme@abaete.com.br,DECISÃO Nº 36,05/03/2020,NaN
3,PRIME TÁXI AIR LTDA,NaN,NaN,23.568.370/0001-25,"TÁXI-AÉREO, TRANSPORTE AÉREO NÃO REGULAR, SERV...","AL. RIO NEGRO, 585 - BLOCO A - CJ.95 - SALA E,...",(11) 41958307 / (11) 41958308,mmatta@primefractionclub.com.br,DECISÃO Nº 237,17/12/2020,18/12/2025
0,RIMA - RIO MADEIRA AEROTÁXI LTDA.,RIM,0R,04.778.630/0001-42,"LIGAÇÃO AÉREA SISTEMÁTICA REGULAR, TÁXI-AÉREO","AV. LAURO SODRÉ, Nº 6490 - AEROPORTO INTERNACI...",(69) 3225-8000,NaN,DECISÃO Nº 116,26/07/2017,27/07/2022
1,RTS - ROTA DO SOL TÁXI AÉREO LTDA.,CKP,NaN,01.904.715/0001-31,"LIGAÇÃO AÉREA SISTEMÁTICA REGULAR, SERVIÇOS AÉ...","PRAÇA EDUARDO GOMES, S/Nº, TERMINAL DE AVIAÇÃO...",(85) 3272-3555,quadros@voerotadosol.com,DECISÃO Nº 21,13/02/2019,15/02/2024


In [30]:
#*

icao_none_reg_co = ['ABJ', 'RIM', 'CKP']

df_vra[df_vra.icao_empresa_aerea.isin(icao_none_reg_co)]


,icao_empresa_aerea,numero_voo,codigo_autorizacao,codigo_tipo_linha,icao_aerodromo_origem,icao_aerodromo_destino,partida_prevista,partida_real,chegada_prevista,chegada_real,situacao_voo,codigo_justificativa


#Test subqueries 

q = """
        WITH aeros_volume_origem AS(
        SELECT 
           ae.icao AS icao_airport_o,
           v.icao_empresa_aerea AS icao_empresa_aerea_o,
           COUNT(*) AS volume_empresa_o,
           strftime('%Y', COALESCE(partida_prevista, partida_real)) as year

        FROM df_vra v
        LEFT JOIN df_aerodromos ae ON v.icao_aerodromo_origem = ae.icao
        WHERE icao_empresa_aerea_o IS NOT NULL AND icao_airport_o IS NOT NULL
        GROUP BY icao_airport_o, icao_empresa_aerea_o, year
        ),
        
        aeros_volume_destino AS(
            SELECT 
               ae.icao AS icao_airport_d,
               v.icao_empresa_aerea AS icao_empresa_aerea_d,
               COUNT(*) AS volume_empresa_d,
               strftime('%Y', COALESCE(partida_prevista, partida_real)) as year

            FROM df_vra v
            LEFT JOIN df_aerodromos ae ON v.icao_aerodromo_destino = ae.icao
            WHERE icao_empresa_aerea_d IS NOT NULL AND icao_airport_d IS NOT NULL
            GROUP BY icao_airport_d, icao_empresa_aerea_d, year
            )
        
        
            SELECT 
                    a.icao AS icao_aero,
                    a_orig.icao_empresa_aerea_o,
                    a_orig.volume_empresa_o AS quantidade_origem,
                    a_dest.volume_empresa_d AS quantidade_destino,
                    a_orig.volume_empresa_o + a_dest.volume_empresa_d AS total_voos,
                    a_dest.year

            FROM df_aerodromos a
            JOIN aeros_volume_origem a_orig ON a.icao = a_orig.icao_airport_o
            JOIN aeros_volume_destino a_dest ON a.icao = a_dest.icao_airport_d
                                                AND a_orig.icao_empresa_aerea_o = a_dest.icao_empresa_aerea_d
                                                AND a_orig.year = a_dest.year
"""

pysqldf(q)


In [33]:
#'Para cada aeroporto trazer a companhia aérea com maior atuação no ano'

q2 = """
    WITH aeros_volume_origem AS(
        SELECT 
           ae.icao AS icao_airport_o,
           v.icao_empresa_aerea AS icao_empresa_aerea_o,
           COUNT(*) AS volume_empresa_o,
           strftime('%Y', COALESCE(partida_prevista, partida_real)) as year

        FROM df_vra v
        LEFT JOIN df_aerodromos ae ON v.icao_aerodromo_origem = ae.icao
        WHERE icao_empresa_aerea_o IS NOT NULL AND icao_airport_o IS NOT NULL
        GROUP BY icao_airport_o, icao_empresa_aerea_o, year
        ),
        
        aeros_volume_destino AS(
            SELECT 
               ae.icao AS icao_airport_d,
               v.icao_empresa_aerea AS icao_empresa_aerea_d,
               COUNT(*) AS volume_empresa_d,
               strftime('%Y', COALESCE(partida_prevista, partida_real)) as year

            FROM df_vra v
            LEFT JOIN df_aerodromos ae ON v.icao_aerodromo_destino = ae.icao
            WHERE icao_empresa_aerea_d IS NOT NULL AND icao_airport_d IS NOT NULL
            GROUP BY icao_airport_d, icao_empresa_aerea_d, year
            ),
        
        aeros_total AS (
            SELECT 
                    a.name AS nome_aero,
                    a.icao AS icao_aero,
                    a_orig.icao_empresa_aerea_o,
                    a_orig.volume_empresa_o AS quantidade_origem,
                    a_dest.volume_empresa_d AS quantidade_destino,
                    a_orig.volume_empresa_o + a_dest.volume_empresa_d AS total_voos,
                    a_dest.year

            FROM df_aerodromos a
            JOIN aeros_volume_origem a_orig ON a.icao = a_orig.icao_airport_o
            JOIN aeros_volume_destino a_dest ON a.icao = a_dest.icao_airport_d
                                                AND a_orig.icao_empresa_aerea_o = a_dest.icao_empresa_aerea_d
                                                AND a_orig.year = a_dest.year
            ),
            
        row_table AS (
            SELECT 
                    *,
                    ROW_NUMBER() OVER(PARTITION BY a_t.icao_aero, a_t.year ORDER BY a_t.total_voos DESC) AS row
            
            FROM aeros_total AS a_t
            )
        
        SELECT 
                r.nome_aero,
                r.icao_aero,
                ac.razao_social,
                r.quantidade_origem,
                r.quantidade_destino,
                r.total_voos
                
        FROM row_table r
        JOIN df_air_cia ac ON r.icao_empresa_aerea_o = ac.icao 
        WHERE row = 1
        ORDER BY r.total_voos DESC
        ;
    """

pysqldf(q2)


,nome_aero,icao_aero,razao_social,quantidade_origem,quantidade_destino,total_voos
0,Viracopos/Campinas International Airport,SBKP,AZUL LINHAS AÉREAS BRASILEIRAS S/A,41761,41662,83423
1,São Paulo–Guarulhos International Airport,SBGR,TAM LINHAS AÉREAS S.A.,33173,33181,66354
2,Tancredo Neves International Airport (Confins ...,SBCF,AZUL LINHAS AÉREAS BRASILEIRAS S/A,20987,20952,41939
3,Recife/Guararapes–Gilberto Freyre Internationa...,SBRF,AZUL LINHAS AÉREAS BRASILEIRAS S/A,20492,20499,40991
4,Brasília International Airport (Presidente J. ...,SBBR,TAM LINHAS AÉREAS S.A.,16174,16170,32344
...,...,...,...,...,...,...
115,Queen Beatrix International Airport,TNCA,AZUL LINHAS AÉREAS BRASILEIRAS S/A,2,2,4
116,Augusto C. Sandino International Airport,MNMG,AZUL LINHAS AÉREAS BRASILEIRAS S/A,1,1,2
117,Monseñor Óscar Arnulfo Romero International Ai...,MSLP,AZUL LINHAS AÉREAS BRASILEIRAS S/A,1,1,2
118,Domingo Faustino Sarmiento Airport,SANU,GOL LINHAS AÉREAS S.A. (EX- VRG LINHAS AÉREAS ...,1,1,2
